In [40]:
import os
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))

import glob
import time
import math
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio

from osgeo import ogr
from osgeo import gdal

import matplotlib.pyplot as plt


In [41]:
csv_folder = f'{BASE_DIR}/csv2019novto2020nov'
csv_folder

'/Users/biplovbhandari/UAH/GRA_Work/AQ_Downscaling/csv2019novto2020nov'

In [42]:
csv_files = glob.glob(f'{csv_folder}/*.csv')
csv_files.sort()
# csv_files = csv_files[:1]
len(csv_files)

373

In [43]:
output_path_shapefile = f'{BASE_DIR}/labels_shapefile'
Path(f'{output_path_shapefile}').mkdir(parents=True, exist_ok=True)
output_path_shapefile

'/Users/biplovbhandari/UAH/GRA_Work/AQ_Downscaling/labels_shapefile'

In [44]:
output_path_test = f'{BASE_DIR}/test_label'
Path(f'{output_path_test}').mkdir(parents=True, exist_ok=True)
output_path_test

'/Users/biplovbhandari/UAH/GRA_Work/AQ_Downscaling/test_label'

In [45]:
output_path_raster = f'{BASE_DIR}/labels_raster'
Path(f'{output_path_raster}').mkdir(parents=True, exist_ok=True)
output_path_raster

'/Users/biplovbhandari/UAH/GRA_Work/AQ_Downscaling/labels_raster'

In [58]:
def grid(limit, dx, dy, indata, inlat, inlon):
#     dx = gsize
#     dy = gsize

    min_lat = limit[0]
    max_lat = limit[1]
    min_lon = limit[2]
    max_lon = limit[3]

    xdim = int(1 + ((max_lon - min_lon) / dx))
    ydim = int(1 + ((max_lat - min_lat) / dy))

    avgtau = np.full([xdim, ydim], -1.)
    count  = np.zeros((xdim, ydim))
    sumtau = np.zeros((xdim, ydim))

    for ii in range(len(indata)):
        if (inlat[ii] >= min_lat and inlat[ii] <= max_lat and inlon[ii] >= min_lon and inlon[ii] <= max_lon):
            i = int((inlon[ii] - min_lon) / dx)
            j = int((max_lat - inlat[ii]) / dy)

            # no data value
            if indata[ii] != -999.9:
                sumtau[i, j] += indata[ii]
                count[i, j] += 1

    for i in range(xdim):
        for j in range(ydim):
            if count[i, j] > 0:
                avgtau[i, j] = sumtau[i, j] / (count[i, j] * 1.)
            else:
                avgtau[i, j] = 0.

    return avgtau

In [59]:
# min_lat = min_y = 5.431855772
# max_lat = max_y = 20.625
# min_lon = min_x = 97.03125
# max_lon = max_x = 105.675625164
# gsize = 0.045

# these value comes from the upscaled raster (Taken from QGIS)
# min_lat = min_y = 5.366368082
# max_lat = max_y = 20.625000000
# min_lon = min_x = 97.03125
# max_lon = max_x = 105.806600545

min_lat = min_y = 5.3762771618097744
max_lat = max_y = 20.6250000000000000
min_lon = min_x = 97.0312500000000000
max_lon = max_x = 105.7834755343458681

limit = [min_lat, max_lat, min_lon, max_lon]

# dx = 0.078125
# dy = 0.0625
dx = 0.045
dy = 0.045

In [77]:
start = time.time()

completed = 0

for index, csv_file in enumerate(csv_files):

    date = (csv_file.split(f'{csv_folder}/')[1]).split('.csv')[0]
    print(date)
    Path(f'{output_path_shapefile}/{date}').mkdir(parents=True, exist_ok=True)

    df = pd.read_csv(csv_file, index_col='ground_datetime')
    df.index = pd.to_datetime(df.index)

    _df = df[df.index.strftime('%Y-%m-%d') == date]    
    timed_df = _df.groupby(by='ground_datetime')

    for hour, subgroup in timed_df:
#         grid_df = gpd.read_file(f'{BASE_DIR}/grid/grid_raster_extent_wgs_deg.shp', crs='epsg:4326')
        grid_df = gpd.read_file(f'{BASE_DIR}/grid/grid_thailand_upscaled.shp', crs='epsg:4326')
        indata = subgroup.ground_pm25.tolist()
        inlat  = subgroup.station_lat.tolist()
        inlon  = subgroup.station_lon.tolist()

        avg = grid(limit, dx, dy, indata, inlat, inlon)
        average = avg.flatten()
    #     average = np.flip(average)
        grid_df['average'] = average
        hour_ = str(hour).replace(" ", "_").replace(":", "-")
        grid_df.to_file(f'{output_path_shapefile}/{date}/{hour_}.shp', driver='ESRI Shapefile')

    completed += 1
    if index % 10 == 0:
        print(f'Completed {completed} out of {len(csv_files)}: {round(completed / len(csv_files) * 100, 2)}%')

print(f'Time: {(time.time() - start) / 60.} minutes')


2019-11-23
Completed 1 out of 373: 0.27%
2019-11-24
2019-11-25
2019-11-26
2019-11-27
2019-11-28
2019-11-29
2019-11-30
2019-12-01
2019-12-02
2019-12-03
Completed 11 out of 373: 2.95%
2019-12-04
2019-12-05
2019-12-06
2019-12-07
2019-12-08
2019-12-09
2019-12-10
2019-12-11
2019-12-12
2019-12-13
Completed 21 out of 373: 5.63%
2019-12-14
2019-12-15
2019-12-16
2019-12-17
2019-12-18
2019-12-19
2019-12-20
2019-12-21
2019-12-22
2019-12-23
Completed 31 out of 373: 8.31%
2019-12-24
2019-12-25
2019-12-26
2019-12-27
2019-12-28
2019-12-29
2019-12-30
2019-12-31
2020-01-01
2020-01-02
Completed 41 out of 373: 10.99%
2020-01-03
2020-01-04
2020-01-05
2020-01-06
2020-01-07
2020-01-08
2020-01-09
2020-01-10
2020-01-11
2020-01-12
Completed 51 out of 373: 13.67%
2020-01-13
2020-01-14
2020-01-15
2020-01-16
2020-01-17
2020-01-18
2020-01-19
2020-01-20
2020-01-21
2020-01-22
Completed 61 out of 373: 16.35%
2020-01-23
2020-01-24
2020-01-25
2020-01-26
2020-01-27
2020-01-28
2020-01-29
2020-01-30
2020-01-31
2020-02-01


In [78]:
start = time.time()

completed = 0

for index, csv_file in enumerate(csv_files):
    date = (csv_file.split(f'{csv_folder}/')[1]).split('.csv')[0]
    Path(f'{output_path_raster}/{date}').mkdir(parents=True, exist_ok=True)
    
#     shp = ogr.Open(f'{output_path_shapefile}/{file_name}.shp')
#     print(shp)

    all_files = glob.glob(f'{output_path_shapefile}/{date}/*.shp')
    all_files = [file.split(f'{output_path_shapefile}/{date}/')[1] for file in all_files]
    unique_hours = [file.split('.shp')[0] for file in all_files]

    for hour in unique_hours:    
        # see options here: https://gdal.org/python/
        ds = gdal.Rasterize(f'{output_path_raster}/{date}/{hour}.tiff', f'{output_path_shapefile}/{date}/{hour}.shp',
                            xRes=dx, yRes=dy, attribute='average',
                            outputBounds=[min_x, min_y, max_x, max_y],
                            outputType=gdal.GDT_Float32)
        ds = None

    completed += 1
    if index % 15 == 0:
        print(f'Completed {completed} out of {len(csv_files)}: {round(completed / len(csv_files) * 100, 2)}%')

print(f'Time: {(time.time() - start) / 60.} minutes')



Completed 1 out of 373: 0.27%
Completed 16 out of 373: 4.29%
Completed 31 out of 373: 8.31%
Completed 46 out of 373: 12.33%
Completed 61 out of 373: 16.35%
Completed 76 out of 373: 20.38%
Completed 91 out of 373: 24.4%
Completed 106 out of 373: 28.42%
Completed 121 out of 373: 32.44%
Completed 136 out of 373: 36.46%
Completed 151 out of 373: 40.48%
Completed 166 out of 373: 44.5%
Completed 181 out of 373: 48.53%
Completed 196 out of 373: 52.55%
Completed 211 out of 373: 56.57%
Completed 226 out of 373: 60.59%
Completed 241 out of 373: 64.61%
Completed 256 out of 373: 68.63%
Completed 271 out of 373: 72.65%
Completed 286 out of 373: 76.68%
Completed 301 out of 373: 80.7%
Completed 316 out of 373: 84.72%
Completed 331 out of 373: 88.74%
Completed 346 out of 373: 92.76%
Completed 361 out of 373: 96.78%
Time: 2.7582204500834147 minutes
